In [2]:
import os

from pathlib import Path

import pandas as pd

from star_parser import StarParser, StarTabDf

In [3]:
data_folder = Path(r'/mnt/DATA/andrea/AM_20211112_NeCen_BRCA1AUb/cryolo_pick')
file_pattern = '*fractions.star'
mic_prefix = ""
mic_prefix_remove = 0
mic_suffix_add = ""
mic_suffix_remove = ""
star_out = "cryolo_pick_all.star"

In [4]:
starfiles = data_folder.glob(f'{file_pattern}')
collated = []
for star in starfiles:
    data = StarParser(star).parse()['data_']
    mic_name = mic_prefix + star.name.replace(mic_suffix_remove,"").replace('.star',f'{mic_suffix_add}.mrc')
    data.fill_column('MicrographName', mic_name, create=True, store=True)
    data.remove_columns(['ClassNumber',	'AnglePsi',	'AutopickFigureOfMerit'], store=True)
    new_order = [data.columns()[-1]] + data.columns()[:-1]
    data.reorder_columns(new_order, store=True)
    collated.append(data.to_df())

In [5]:
merged = pd.concat(collated)
collated_star = StarTabDf(merged)

In [6]:
collated_star.to_df()
collated_star.columns()

['MicrographName', 'CoordinateX', 'CoordinateY']

In [7]:
import copy
x = copy.deepcopy(collated_star)
x.df

,MicrographName,CoordinateX,CoordinateY
0,FoilHole_12383532_Data_12377348_12377350_20211...,993.0,3944.0
1,FoilHole_12383532_Data_12377348_12377350_20211...,690.0,3887.0
2,FoilHole_12383532_Data_12377348_12377350_20211...,2398.0,3881.0
3,FoilHole_12383532_Data_12377348_12377350_20211...,2231.0,3728.0
4,FoilHole_12383532_Data_12377348_12377350_20211...,550.0,3625.0
...,...,...,...
1380792,FoilHole_12389199_Data_12377348_12377350_20211...,4208.0,153.0
1380793,FoilHole_12389199_Data_12377348_12377350_20211...,1121.0,116.0
1380794,FoilHole_12389199_Data_12377348_12377350_20211...,2470.0,107.0
1380795,FoilHole_12389199_Data_12377348_12377350_20211...,2773.0,93.0


In [8]:
x.df.rename(columns={"CoordinateX":"CoordinateY", "CoordinateY":"CoordinateX"}, inplace=True)
y = x.df
y['CoordinateX'] = 11520 - y['CoordinateX'].astype(float)
y['CoordinateY'] = 8184 - y['CoordinateY'].astype(float)

In [9]:
y['CoordinateY'] = y['CoordinateY'].astype(int).astype(str)
y['CoordinateX'] = y['CoordinateX'].astype(int).astype(str)

In [10]:
collated_star.df = y
collated_star.to_df()

,MicrographName,CoordinateY,CoordinateX
0,FoilHole_12383532_Data_12377348_12377350_20211...,7191,7576
1,FoilHole_12383532_Data_12377348_12377350_20211...,7494,7633
2,FoilHole_12383532_Data_12377348_12377350_20211...,5786,7639
3,FoilHole_12383532_Data_12377348_12377350_20211...,5953,7792
4,FoilHole_12383532_Data_12377348_12377350_20211...,7634,7895
...,...,...,...
1380792,FoilHole_12389199_Data_12377348_12377350_20211...,3976,11367
1380793,FoilHole_12389199_Data_12377348_12377350_20211...,7063,11404
1380794,FoilHole_12389199_Data_12377348_12377350_20211...,5714,11413
1380795,FoilHole_12389199_Data_12377348_12377350_20211...,5411,11427


In [12]:
split = collated_star.to_df().groupby('MicrographName')
split_dir = data_folder/"swapped"
if not split_dir.exists():
    split_dir.mkdir()
for fname, data in split:
    x = StarTabDf(data)
    fname = fname.replace('.mrc','.star')
    with open(split_dir/fname, 'w') as f:
        f.write(x.to_star())